In [11]:
!pip install openpyxl pyperclip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import time
import webbrowser
import pyperclip
from openpyxl import load_workbook
import pyautogui

caminho_arquivo = "C:\\Users\\lz6321\\OneDrive - grendene.com.br\\Área de Trabalho\\Migração_Arquivos_Não_Baixados.xlsx"

wb = load_workbook(caminho_arquivo)
ws = wb.active  

hiperlinks = []
nomes_processados = set()  

for row in ws.iter_rows(min_col=1, max_col=1, min_row=2, max_row=16, values_only=False): 
    for cell in row:
        if cell.hyperlink and cell.value not in nomes_processados:
            hiperlinks.append((cell.row, cell.value, cell.hyperlink.target))
            nomes_processados.add(cell.value)  

def acessar_e_copiar_url(link):
    webbrowser.open(link) 
    time.sleep(7)  
    # Botão de Edição
    pyautogui.click(x=-719, y=42) 
    time.sleep(7)
    # Espaço de edição
    pyautogui.click(x=-982, y=279) 
    time.sleep(7)

    pyautogui.hotkey('ctrl', 'a')  
    time.sleep(1)
    pyautogui.hotkey('ctrl', 'c')  
    time.sleep(1)
    # Localiza o Word
    pyautogui.click(x=985, y=874)
    time.sleep(3)
    # Abre o Word
    pyautogui.click(x=263, y=215) 
    time.sleep(7)
    pyautogui.hotkey('ctrl', 'v') 
    time.sleep(12)

for linha, texto, link in hiperlinks:
    print(f"Acessando o link {link}...")
    print(f"Título: {texto}")

    # Copiar o conteúdo da página
    acessar_e_copiar_url(link)
    
    # Fecha Arquivo
    pyautogui.click(x=1419, y=17) 
    time.sleep(4)

    # Título da página
    pyperclip.copy(texto)  
    pyautogui.hotkey('ctrl', 'v')  
    time.sleep(8)

    pyautogui.click(x=691, y=664) 
    time.sleep(12)

print("Arquivos Word criados com sucesso.")

Acessando o link https://grendenecombr.sharepoint.com/sites/PC-TI/Documentaes%20de%20Sistemas1/WSGCP0066%20Relat%C3%B3rio%20Demonstrativo%20Custo%20de%20Produ%C3%A7%C3%A3o%20-%20QlikView.aspx...
Título: WSGCP0066 Relatório Demonstrativo Custo de Produção - QlikView
Acessando o link https://grendenecombr.sharepoint.com/sites/PC-TI/Documentaes%20de%20Sistemas1/SPED%20Fiscal.aspx...
Título: SPED Fiscal
Acessando o link https://grendenecombr.sharepoint.com/sites/PC-TI/Documentaes%20de%20Sistemas1/WGPD241a%20%E2%80%93%20Descontos%20por%20Processo%20Export%20-%20Web.aspx...
Título: WGPD241a – Descontos por Processo Export - Web
Acessando o link https://grendenecombr.sharepoint.com/sites/PC-TI/Documentaes%20de%20Sistemas1/WGAC528%20-%20Carga%20Cotas%20Venda%20por%20Representante.aspx...
Título: WGAC528 - Carga Cotas Venda por Representante
Acessando o link https://grendenecombr.sharepoint.com/sites/PC-TI/Documentaes%20de%20Sistemas1/NFS-e%20-%20Processo%20de%20Emiss%C3%A3o%20de%20Nota%20Fisca

In [4]:
import time
import pyautogui

time.sleep(8)  # Dá tempo de posicionar o mouse
print(pyautogui.position())  # Mostra as coordenadas do cursor


Point(x=985, y=874)


In [48]:
import os
import shutil
import pandas as pd
import re
import uuid

# Carregar a planilha
arquivo_excel = "C:\\Users\\lz6321\\Downloads\\migracao2.xlsx"
df = pd.read_excel(arquivo_excel)

# Caminho base onde os arquivos estão localizados e onde as pastas serão criadas
caminho_base = "C:\\Users\\lz6321\\OneDrive - grendene.com.br"

# Função para remover caracteres inválidos e substituir barras
def normalizar_nome(nome):
    if nome is None or nome.strip() == "":
        return "Desconhecido"  # Define um nome padrão para evitar erro
    
    nome = nome.strip()  # Remove espaços extras
    nome = nome.replace("/", "_")  # Substitui barras por underscore
    return re.sub(r'[<>:"\\|?*]', '_', nome)  # Substitui caracteres inválidos

# Função para garantir um nome único
def garantir_nome_unico(nome_pasta):
    return f"{nome_pasta}_{uuid.uuid4().hex[:8]}"  # Adiciona um identificador único

def encontrar_arquivo(nome_base):
    """Verifica se o arquivo existe no caminho base considerando diferentes extensões."""
    for ext in [".docx", ".doc"]:
        arquivo_caminho = os.path.join(caminho_base, nome_base + ext)
        if os.path.exists(arquivo_caminho):
            return arquivo_caminho
    return None

def organizar_arquivos():
    for _, row in df.iterrows():
        # Normalizar e limpar espaços extras
        nome_area = normalizar_nome(str(row["Nivel 1 - Área"]).strip())
        nome_modulo = normalizar_nome(str(row["Nivel 2 - Módulo"]).strip())
        
        pasta_c = os.path.join(caminho_base, nome_area)
        pasta_d = os.path.join(pasta_c, nome_modulo)

        # Imprime os caminhos gerados para verificar
        print(f"Verificando pasta: {pasta_d}")

        # Criar as pastas se não existirem
        os.makedirs(pasta_d, exist_ok=True)

        nome_arquivo = normalizar_nome(str(row["Nivel 3 - Programa"]).strip())

        # Tenta encontrar o arquivo com diferentes extensões
        arquivo_origem = encontrar_arquivo(nome_arquivo)

        # Verifica se o arquivo foi encontrado
        if arquivo_origem:
            arquivo_destino = os.path.join(pasta_d, os.path.basename(arquivo_origem))
            try:
                # Verifica se o arquivo já existe no destino
                if os.path.exists(arquivo_destino):
                    print(f"O arquivo {arquivo_destino} já existe no destino. Pulando...")
                else:
                    shutil.move(arquivo_origem, arquivo_destino)
                    print(f"Movido: {arquivo_origem} -> {arquivo_destino}")
            except Exception as e:
                print(f"Erro ao mover o arquivo {arquivo_origem}: {e}")
        else:
            print(f"Arquivo não encontrado para: {nome_arquivo} no caminho {caminho_base}")

if __name__ == "__main__":
    organizar_arquivos()

Verificando pasta: C:\Users\lz6321\OneDrive - grendene.com.br\Cadastros\Cadastros Gerais
Arquivo não encontrado para: Acessando o Flex via RDP no caminho C:\Users\lz6321\OneDrive - grendene.com.br
Verificando pasta: C:\Users\lz6321\OneDrive - grendene.com.br\Cadastros\Cadastros Gerais
Arquivo não encontrado para: Funcionalidades Básicas de Telas no caminho C:\Users\lz6321\OneDrive - grendene.com.br
Verificando pasta: C:\Users\lz6321\OneDrive - grendene.com.br\Cadastros\Cadastros Gerais
Arquivo não encontrado para: Módulo Cadastros Gerais no caminho C:\Users\lz6321\OneDrive - grendene.com.br
Verificando pasta: C:\Users\lz6321\OneDrive - grendene.com.br\Cadastros\Cadastros Gerais
Arquivo não encontrado para: Acesso ao banco de Historico no ERP Totvs no caminho C:\Users\lz6321\OneDrive - grendene.com.br
Verificando pasta: C:\Users\lz6321\OneDrive - grendene.com.br\Comercial\Vendas
Arquivo não encontrado para: CD0708 - Manutenção Representante no caminho C:\Users\lz6321\OneDrive - grendene

In [34]:
import os
caminho_base = "C:\\Users\\lz6321\\OneDrive - grendene.com.br"

if os.path.exists(caminho_base):
    print("Caminho base encontrado!")
else:
    print("Erro: Caminho base NÃO encontrado. Verifique se está correto.")


Caminho base encontrado!
